In [1]:
# 导入可能需要的包
import zipfile
import os 
import sys
import nltk
import pandas as pd
import numpy as np

In [2]:
# os.path.dirname('Part1.ipynb')
# os.getcwd()
# par_dir = os.path.pardir
# cur_path = os.path.join()

In [3]:
# 获取当前目录
# os.getcwd() #绝对地址
# os.path.abspath('.') #绝对地址
# os.path.abspath(os.path.dirname(__file__)) # 在jupyter里不能用

In [4]:
# 获取上一级目录
# os.path.abspath('..')
# os.path.abspath(os.path.dirname(os.path.dirname(__file__)))
# os.path.abspath(os.path.dirname(os.getcwd())) # 其实貌似是同os.path.dirname(os.getcwd()的
# os.path.abspath(os.path.join(os.getcwd(), ".."))
# os.path.join(os.getcwd(), "..")

# 由于os.path.dirname这个方法很直观，也感觉会很常用，所以以后统一使用该方法和os.getcwd()的组合获取当前和上一级目录名
cur_par_dir = os.path.dirname(os.getcwd())
# cur_par_dir

In [5]:
# 获取目录下的文件
os.listdir(cur_par_dir)
# os.listdir(os.path.join(par_dir,'source_data'))

['code', 'data', 'source_data']

In [6]:
## 0 数据准备
source_data_path = os.path.join(cur_par_dir,'source_data')
obj_par = {
    'unzipped_pack_paths':[os.path.join(source_data_path,path) for path in os.listdir(source_data_path)],
    'data_target':os.path.join(cur_par_dir,'data')
}


def show_par(detail = False, entity = ''):
    if detail:
        print(obj_par[entity])
    else:
        print(obj_par.keys())
    
# show_par()
unzipped_pack_paths  = obj_par['unzipped_pack_paths']
target = obj_par['data_target']
# unzipped_pack_paths 
target
unzipped_pack_paths

os.path.splitext(unzipped_pack_paths[1])[1]

'.csv'

In [7]:
def unzip_data(unzipped_pack_paths,target):
    for path in unzipped_pack_paths:
        if os.path.splitext(path)[1] == '.zip':
            with zipfile.ZipFile(path) as package:
                try:
                    package.extractall(target)
                except zipfile.BadZipFile:
                    print("unzip error")
                    
def show_datafile(data_path = target):
    print(os.listdir(data_path))

In [8]:
if not os.path.exists(target):
    os.makedirs(target)


unzip_data(unzipped_pack_paths, target)


In [9]:
# show_datafile()
# show_par(detail = True,entity =  'data_target')
os.path.join(obj_par['data_target'],'labeledTrainData.tsv')

'D:\\CodeFiled\\NLP_Learning\\Kaggle1\\working_lab\\data\\labeledTrainData.tsv'

In [10]:
## 数据处理
# 导入

# 没有delimiter="\t" 是读不了这个数据的
raw_labeled_review = pd.read_csv(os.path.join(obj_par['data_target'],'labeledTrainData.tsv'), delimiter="\t", quoting=3)
raw_labeled_review.head

row_num, col_num = raw_labeled_review.shape
# raw_labeled_review.columns

In [11]:
# 数据清洁1：网上爬的，清一下HMTL标签。根据字符串对象创建一个BeautifulSoup对象可以直接提取出不含标签的文字。
from bs4 import BeautifulSoup

sample1 = raw_labeled_review['review'][0]

sample1_soup = BeautifulSoup(sample1)

sample1
sample1_soup.get_text()

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.The actual feature film bit when it finally starts is only on for 2

In [12]:
# 语义精炼
# 数据处理1：为了方便，在这里不将特殊的标点当做词汇，直接去除，即只保留是字符的，不是字母的字符用空格替换
import re
pattern = '[^a-zA-Z]'

sample2 = sample1_soup.get_text()
re.sub(pattern, ' ', sample2)

sample2 = sample2.lower()
sample2 = sample2.split(" ")


In [13]:
# 数据处理2：去除掉语义信息少的高频词
from nltk.corpus import stopwords

stopword_corpus = stopwords.words('English')
# 把字符串变成小写后，分离成词表，再与stopwords的词表对比，得到不含stopwords的词表

sample3 = [w for w in sample2 if not w in stopword_corpus]

# " ".join(sample3)
# sample3

stopword_corpus = set(stopword_corpus)# 加快速度

In [14]:
# 数据处理函数，只针对单条数据，后面做循环就好
def review_proc(review):
    rev_HTML_review = BeautifulSoup(review).get_text()
    
    rev_pun = re.sub('[^a-zA-Z]', " ", rev_HTML_review)
    
    lowor_review_list = rev_pun.lower().split(" ")
    
    rev_stop_review_list = [w for w in lowor_review_list if not w in stopword_corpus]
    
    return ' '.join(rev_stop_review_list)

In [15]:
processed_reviews = []

nums, _ = raw_labeled_review.shape

for i in range(0, nums):
    processed_reviews.append(review_proc(raw_labeled_review['review'][i]))

# processed_reviews

C:\Users\FlashBlack7\AppData\Local\Temp\ipykernel_49008\3990676818.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  rev_HTML_review = BeautifulSoup(review).get_text()


In [16]:
# 2 Bags of words：思想，基于数据和常用词给出一个vocabulary,对应一个高维的稀疏向量，对于一个句子，统计里面存在于vocalbulary里词汇的个数，并记录在对应分量位置

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(
    stop_words = None,
    analyzer= "word",
    tokenizer = None,
    max_features = 5000)

#_.fit_transform() 方法接受一个文本数组，其中每个元素都是一篇文章或一条消息。最后返回一个稀疏矩阵，每行为一样本
train_data = vectorizer.fit_transform(processed_reviews) 

# train_data = train_data.toarray() #现在貌似会自动转成numpy的数组了
# type(train_data)
# dist = np.sum(train_data, axis=0)
# dist.shape

In [17]:
## plus:显示当下定义的函数
def func1():
    pass


def show_funcs():
    print([name for name in dir() if callable(locals().get(name)) and isinstance(locals().get(name), type(func1))])


In [18]:
# 3 模型训练
# from sklearn.ensemble import RandomForestClassifier

# forest = RandomForestClassifier(n_estimators = 100,verbose=True)
# forest = RandomForestClassifier(n_estimators = 100)


# forest  = forest.fit(train_data, raw_labeled_review['sentiment'],verbose=True)
# forest  = forest.fit(train_data, raw_labeled_review['sentiment'])
# raw_labeled_review['sentiment']


In [19]:
# import logging

# logger = logging.getLogger(__name__)
# logger.setLevel(logging.INFO)

# formatter = logging.Formatter(' %(message)s')

# console_handler = logging.StreamHandler()
# console_handler.setLevel(logging.INFO)
# console_handler.setFormatter(formatter)

# logger.addHandler(console_handler)

# for i in range(10):
#     logger.debug('Iteration %d', i)
#     logger.info('Iteration %d', i)

In [20]:
## sklearn的模型序列化有两种方法:
# 二进制化保存成文件
# import pickle


# with open(os.path.join(os.getcwd(),'RandomForest.pkl'), 'wb') as file:
#     pickle.dump(forest, file)

# # 加载模型
# with open(os.path.join(os.getcwd(),'RandomForest.pkl'), 'rb') as file:
#     forest = pickle.load(file)

# 压缩到磁盘
import joblib

# joblib.dump(forest, os.path.join(os.getcwd(),'RandomForest.joblib' ))

forest = joblib.load(os.path.join(os.getcwd(),'RandomForest.joblib' ))
# 以上两个方法根据搜索结果，joblib更适合大数据，因此以后默认采用后者

In [21]:
## 4 在测试集上运行模型
# 解压
# os.listdir(obj_par['data_target']))
raw_test_reviews = pd.read_csv(os.path.join(obj_par['data_target'],'testData.tsv'), delimiter="\t", quoting=3)

rows, _ = raw_test_reviews.shape

clean_text_reviews = []


# raw_test_reviews.head

In [22]:
for i in range(0, rows):
    clean_text_reviews.append(review_proc(raw_test_reviews["review"][i]))
    if (i+1) % 1000 == 0:
        print("iteration: %d",i+1)

test_data = vectorizer.transform(clean_text_reviews)

C:\Users\FlashBlack7\AppData\Local\Temp\ipykernel_49008\3990676818.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  rev_HTML_review = BeautifulSoup(review).get_text()


iteration: %d 1000
iteration: %d 2000
iteration: %d 3000
iteration: %d 4000
iteration: %d 5000
iteration: %d 6000
iteration: %d 7000
iteration: %d 8000
iteration: %d 9000
iteration: %d 10000
iteration: %d 11000
iteration: %d 12000
iteration: %d 13000
iteration: %d 14000
iteration: %d 15000
iteration: %d 16000
iteration: %d 17000
iteration: %d 18000
iteration: %d 19000
iteration: %d 20000
iteration: %d 21000
iteration: %d 22000
iteration: %d 23000
iteration: %d 24000
iteration: %d 25000


In [32]:
result = forest.predict(test_data)

submission = pd.DataFrame({'id': raw_test_reviews['id'], 'sentiment':result})
submission.to_csv('Bag_of_Words_model.csv', index=False, quoting=3 )

,id,sentiment
0,"""12311_10""",1
1,"""8348_2""",0
2,"""5828_4""",1
3,"""7186_2""",1
4,"""12128_7""",1
...,...,...
24995,"""2155_10""",1
24996,"""59_10""",1
24997,"""2531_1""",0
24998,"""7772_8""",1
